In [1]:
!pip install streamlit tensorflow torch transformers pillow


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ----------- ---------------------------- 2.6/8.7 MB 16.7 MB/s eta 0:00:01
   ----------------------------------- ---- 7.9/8.7 MB 22.1 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 20.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/658.1 kB ? eta -:--:--
   --------------------------------------- 658.1/658.1 kB 24.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---------- ----------------------------- 6.6/25.1 MB 31.0 MB/s eta 0:00:01
   -------------------- ------------------- 12.8/25.1 MB 29.8 MB/s eta 0:00:01
   ------------------------------ --------- 19.4/25.1 MB 30.6 MB/s eta 0:00:01
   ---------------------------------------- 25.1/25.1 MB 28.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ------------------------------------- -- 6.6/6.9 MB 31.0 MB/s eta 0:00:01
   ---------------

In [ ]:
# Import necessary libraries
import streamlit as st
from transformers import pipeline
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.utils import get_file
from PIL import Image
import numpy as np
import sys
import os

# Ensure UTF-8 Encoding
sys.stdout.reconfigure(encoding='utf-8')
os.environ["PYTHONIOENCODING"] = "utf-8"

# Load models with caching to avoid repeated downloads
@st.cache_resource
def load_models():
    # Load MobileNetV2 without progress bar
    model_path = get_file(
        "mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5",
        "https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224.h5",
        cache_subdir='models'
    )
    image_model = tf.keras.applications.MobileNetV2(weights=model_path)

    # Load Hugging Face sentiment analysis model
    sentiment_model = pipeline("sentiment-analysis")

    return image_model, sentiment_model

# Load the models
image_model, sentiment_model = load_models()

# Streamlit UI
st.title("AI Model Integration with Streamlit")

# Sidebar to choose the task
option = st.sidebar.selectbox("Choose a task", ["Image Classification", "Sentiment Analysis"])

if option == "Image Classification":
    uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])
    if uploaded_file is not None:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)

        # Preprocess the image
        img_array = np.array(image.resize((224, 224)))
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)

        # Run inference on the image
        predictions = image_model.predict(img_array)
        decoded = decode_predictions(predictions, top=3)[0]

        # Display predictions
        st.write("Predictions:")
        for pred in decoded:
            st.write(f"{pred[1]}: {round(pred[2] * 100, 2)}%")

elif option == "Sentiment Analysis":
    text_input = st.text_area("Enter text for sentiment analysis")
    if st.button("Analyze"):
        if text_input:
            # Get sentiment predictions
            result = sentiment_model(text_input)
            st.write("Sentiment:", result[0]["label"])
            st.write("Confidence:", round(result[0]["score"] * 100, 2), "%")
